In [4]:
import cv2
import json
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import defaultdict

# --- KONFIGURACJA ---
IMAGE_PATH = 'office_yolo.png'
VIDEO_PATH = 'office_yolo.mp4'
# Przywróciłem pełną listę progów
THRESHOLDS = [0.1, 0.3, 0.5, 0.7] 
OUTPUT_DIR = 'wyniki_yolo'

os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Ładowanie modelu
model = YOLO('yolov8n.pt') 
print(f"Model załadowany. Klasy modelu: {model.names}")

def generate_plot(stats, title, filename):
    """
    Generuje wykres słupkowy z wykryć przy użyciu Matplotlib.
    """
    if not stats:
        return # Nie rysujemy pustych wykresów
    
    # Sortowanie danych malejąco dla czytelności
    sorted_stats = dict(sorted(stats.items(), key=lambda item: item[1], reverse=True))
    
    labels = list(sorted_stats.keys())
    values = list(sorted_stats.values())
    
    plt.figure(figsize=(10, 6)) # Rozmiar wykresu
    bars = plt.bar(labels, values, color='cornflowerblue', edgecolor='black')
    
    # Dodanie wartości liczbowych nad słupkami
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + (max(values)*0.01), int(yval), ha='center', va='bottom', fontweight='bold')
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Klasa Obiektu', fontsize=12)
    plt.ylabel('Liczba Wykryć (Suma)', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Zapis do pliku i zamknięcie
    plt.savefig(filename)
    plt.close()
    print(f"   -> Wykres zapisano: {filename}")

def print_text_stats(stats, prefix):
    """Drukuje prostą tabelkę w konsoli."""
    total = sum(stats.values())
    print(f"\n   --- Statystyki ({prefix}) ---")
    if total == 0:
        print("   Brak wykryć.")
        return
    for cls, count in sorted(stats.items(), key=lambda x: x[1], reverse=True):
        print(f"   {cls:<15}: {count} ({(count/total)*100:.1f}%)")
    print("   -----------------------------")

def process_image(img_path, conf_threshold):
    print(f"\n--- Przetwarzanie ZDJĘCIA (Prog: {conf_threshold}) ---")
    
    results = model.predict(img_path, conf=conf_threshold, save=False)
    result = results[0]
    
    detections_data = []
    stats = defaultdict(int)
    
    for box in result.boxes:
        cls_id = int(box.cls[0])
        cls_name = result.names[cls_id]
        conf = float(box.conf[0])
        bbox = box.xyxy[0].tolist() 
        
        detections_data.append({
            "class_id": cls_id,
            "class_name": cls_name,
            "confidence": round(conf, 4),
            "bbox": [round(x, 2) for x in bbox]
        })
        stats[cls_name] += 1

    # Zapis JSON
    base_name = f"image_res_conf_{conf_threshold}"
    with open(f"{OUTPUT_DIR}/{base_name}.json", 'w') as f:
        json.dump({"detections": detections_data, "stats": dict(stats)}, f, indent=4)
    
    # Zapis Obrazu
    im_array = result.plot()
    cv2.imwrite(f"{OUTPUT_DIR}/{base_name}.jpg", im_array)
    
    # 1. Statystyki Tekstowe
    print_text_stats(stats, "Obraz")
    
    # 2. Statystyki Wykres (Matplotlib)
    plot_filename = f"{OUTPUT_DIR}/{base_name}_chart.png"
    generate_plot(stats, f"Statystyki ZDJĘCIA (Próg: {conf_threshold})", plot_filename)

def process_video(vid_path, conf_threshold):
    print(f"\n--- Przetwarzanie WIDEO (Prog: {conf_threshold}) ---")
    
    cap = cv2.VideoCapture(vid_path)
    if not cap.isOpened():
        print("Błąd wideo.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    base_name = f"video_res_conf_{conf_threshold}"
    out_vid_name = f"{OUTPUT_DIR}/{base_name}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    out = cv2.VideoWriter(out_vid_name, fourcc, fps, (width, height))
    
    video_data = {
        "threshold": conf_threshold,
        "frames": [],
        "total_stats": defaultdict(int)
    }
    
    frame_idx = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        results = model.predict(frame, conf=conf_threshold, verbose=False)
        result = results[0]
        
        frame_detections = []
        for box in result.boxes:
            cls_id = int(box.cls[0])
            cls_name = result.names[cls_id]
            conf = float(box.conf[0])
            bbox = box.xyxy[0].tolist()
            
            frame_detections.append({
                "class_id": cls_id,
                "class_name": cls_name,
                "confidence": round(conf, 4),
                "bbox": [round(x, 2) for x in bbox]
            })
            video_data["total_stats"][cls_name] += 1
            
        video_data["frames"].append({
            "frame_id": frame_idx,
            "detections": frame_detections
        })
        
        out.write(result.plot())
        frame_idx += 1
        if frame_idx % 20 == 0:
            print(f"   Klatka: {frame_idx}/{total_frames}", end='\r')

    cap.release()
    out.release()
    
    # Zapis JSON
    video_data["total_stats"] = dict(video_data["total_stats"])
    with open(f"{OUTPUT_DIR}/{base_name}.json", 'w') as f:
        json.dump(video_data, f, indent=4)
        
    print(f"\n   Zapisano wideo: {out_vid_name}")
    
    # 1. Statystyki Tekstowe
    print_text_stats(video_data["total_stats"], "Wideo - suma wykryć")
    
    # 2. Statystyki Wykres (Matplotlib)
    plot_filename = f"{OUTPUT_DIR}/{base_name}_chart.png"
    generate_plot(video_data["total_stats"], f"Statystyki WIDEO (Próg: {conf_threshold})", plot_filename)


# --- URUCHOMIENIE ---

# Sprawdź czy pliki wejściowe istnieją
if not os.path.exists(IMAGE_PATH) and not os.path.exists(VIDEO_PATH):
    print("BŁĄD: Nie znaleziono plików wejściowych (obrazu lub wideo).")
    print("Upewnij się, że pliki 'office_yolo.png' i 'office_yolo.mp4' są w folderze projektu.")
else:
    # 1. Zdjęcie
    if os.path.exists(IMAGE_PATH):
        for thresh in THRESHOLDS:
            process_image(IMAGE_PATH, thresh)

    # 2. Wideo
    if os.path.exists(VIDEO_PATH):
        for thresh in THRESHOLDS:
            process_video(VIDEO_PATH, thresh)
            
    print("\n✅ Zakończono! Sprawdź folder 'wyniki_yolo' pod kątem obrazków wykresów.")

Model załadowany. Klasy modelu: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard